In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
iids = ['cordex.output.EUR-11.DMI.ECMWF-ERAINT.evaluation.r1i1p1.HIRHAM5.v1.mon.tas.v20140620',
        'cordex.output.EUR-11.GERICS.ECMWF-ERAINT.evaluation.r1i1p1.REMO2015.v1.mon.tas.v20180813']

In [3]:
import xarray as xr
from pangeo_forge_recipes.patterns import pattern_from_file_sequence
from pangeo_forge_recipes.recipes import XarrayZarrRecipe

from pangeo_forge_cordex import logon, recipe_inputs_from_iids, parse_instance_ids


def test_recipe_inputs():
    iid = 'cordex.output.EUR-11.DMI.ECMWF-ERAINT.evaluation.r1i1p1.HIRHAM5.v1.mon.tas.v20140620'
    iid = 'cordex.output.EUR-11.GERICS.ECMWF-ERAINT.evaluation.r1i1p1.REMO2015.v1.mon.tas.v20180813'
    sslcontext = logon()

    recipe_inputs = recipe_inputs_from_iids(iid, sslcontext)

    urls = recipe_inputs[iid]["urls"]
    recipe_kwargs = recipe_inputs[iid]["recipe_kwargs"]
    pattern_kwargs = recipe_inputs[iid]["pattern_kwargs"]

    pattern = pattern_from_file_sequence(urls, "time", **pattern_kwargs)
    recipe = XarrayZarrRecipe(
        pattern, xarray_concat_kwargs={"join": "exact"}, **recipe_kwargs
    )

    recipe_pruned = recipe.copy_pruned()
    run_function = recipe_pruned.to_function()

    run_function()

    ds = xr.open_zarr(recipe.target_mapper, consolidated=True)
    return ds

In [4]:
def test_parsing():
    iids = parse_instance_ids('cordex.output.EUR-11.DMI.ECMWF-ERAINT.evaluation.r1i1p1.HIRHAM5.v1.mon.tas')
    return iids

test_parsing()

['cordex.output.EUR-11.DMI.ECMWF-ERAINT.evaluation.r1i1p1.HIRHAM5.v1.mon.tas.v20140620']

In [5]:
ds = test_recipe_inputs()

logged on: True
Found 1 dataset(s)!
Found 406 timesteps!
http://esgf1.dkrz.de/thredds/fileServer/cordex/cordex/output/EUR-11/GERICS/ECMWF-ERAINT/evaluation/r1i1p1/GERICS-REMO2015/v1/mon/tas/v20180813/tas_EUR-11_ECMWF-ERAINT_evaluation_r1i1p1_GERICS-REMO2015_v1_mon_197902-198012.nc
Estimated size: 283.693312 MB


In [107]:
df.to_csv('s3://euro-cordex/catalog/dummy.csv', index=False)

In [156]:
df = pd.read_csv('s3://euro-cordex/catalog/CORDEX.csv')
df

,project_id,product,CORDEX_domain,institute_id,driving_model_id,experiment_id,member,model_id,rcm_version_id,frequency,variable_id,version,path
0,cordex,output,EUR-11,GERICS,ECMWF-ERAINT,evaluation,r1i1p1,REMO2015,v1,mon,tas,v20180813,s3://euro-cordex/CMIP5/cordex/output/EUR-11/GE...
1,cordex,output,EUR-11,ICTP,MOHC-HadGEM2-ES,rcp26,r1i1p1,RegCM4-6,v1,mon,tas,v20190502,s3://euro-cordex/CMIP5/cordex/output/EUR-11/IC...
2,cordex,output,EUR-11,CLMcom,ICHEC-EC-EARTH,rcp26,r12i1p1,CCLM4-8-17,v1,mon,tas,v20170523,s3://euro-cordex/CMIP5/cordex/output/EUR-11/CL...
3,cordex-reklies,output,EUR-11,GERICS,MIROC-MIROC5,historical,r1i1p1,REMO2015,v1,mon,tas,v20170329,s3://euro-cordex/CMIP5/cordex-reklies/output/E...


In [157]:
import fsspec
import xarray as xr

target = fsspec.get_mapper(df.path[3], anon=True)

In [159]:
ds = xr.open_zarr(target, consolidated=True)

In [160]:
ds

<xarray.Dataset>
Dimensions:                     (rlat: 412, rlon: 424, vertices: 4, time: 695,
                                 bnds: 2)
Coordinates:
    height                      float64 ...
    lat                         (rlat, rlon) float32 dask.array<chunksize=(412, 424), meta=np.ndarray>
    lon                         (rlat, rlon) float32 dask.array<chunksize=(412, 424), meta=np.ndarray>
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.72 21.84
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.04 18.16
  * time                        (time) datetime64[ns] 1948-02-15T12:00:00 ......
Dimensions without coordinates: vertices, bnds
Data variables:
    lat_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    lon_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    rotated_latitude_longitude  int32 ...
    tas                         (time, rlat, rlon) float32 dask.array<chunksize=(143, 412, 424), meta=np.ndarray>
    time_bnds                   (time, bnds) datetime64[ns] dask.array<chunksize=(143, 2), meta=np.ndarray>
Attributes: (12/29)
    CORDEX_domain:                  EUR-11
    Conventions:                    CF-1.4
    cmor_version:                   2.9.1
    comment:                        CORDEX Europe RCM REMO 0.11 deg EUR-11. T...
    contact:                        gerics-cordex@hzg.de
    creation_date:                  2017-02-27T12:05:55Z
    ...                             ...
    realization:                    1
    references:                     http://www.remo-rcm.de/
    source:                         GERICS-REMO2015
    table_id:                       Table mon (Mar 2015) db0b230ff4a2c922671f...
    title:                          GERICS-REMO2015 model output prepared for...
    tracking_id:                    58f18ebe-ace9-4ca0-8af4-36486f05b369

In [165]:
mean = ds.tas.groupby("time.year").mean().mean(("rlon", "rlat"))
mean

<xarray.DataArray 'tas' (year: 58)>
dask.array<mean_agg-aggregate, shape=(58,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
    height   float64 ...
  * year     (year) int64 1948 1949 1950 1951 1952 ... 2001 2002 2003 2004 2005

In [168]:
from dask.distributed import Client
client = Client()

In [169]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53088,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:53106,Total threads: 1
Dashboard: http://127.0.0.1:53113/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:53091,


In [ ]:
%time mean_ = mean.compute()